<a href="https://colab.research.google.com/github/SusanaSernaJaramillo/Business-Intellingene/blob/main/Reto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se cargan las librerias necesarias:

In [ ]:
#Liberias cientificas
import pandas as pd
import numpy as np

#Librerias del arbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Se carga la base de datos y se obtiene información de los datos.

In [ ]:
nxl='/content/drive/MyDrive/Analítica de negocios Notebooks/Analitica Base de datos/2. CollegeAdmisions_Int_M.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0)

In [ ]:
#Mostrar lineas
XDB.head(100)

,Applicant,Edu_Parent1,Edu_Parent2,Gender,White,Asian,HSGPA,SAT/ACT,College,Admitted,Enrolled,College_GPA
0,1,4,4,M,0,1,4.08,1460,Business & Economics,Yes,No,NaN
1,2,3,3,F,0,0,3.59,1180,Math & Science,No,No,NaN
2,3,7,7,M,1,0,4.25,1340,Arts & Letters,Yes,No,NaN
3,4,6,6,M,0,1,3.00,1190,Business & Economics,No,No,NaN
4,5,6,7,F,1,0,3.90,1290,Business & Economics,No,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,6,5,M,1,0,2.95,1260,Business & Economics,No,No,NaN
96,97,7,6,F,1,0,3.95,1250,Business & Economics,Yes,No,NaN
97,98,6,6,F,1,0,3.71,1190,Math & Science,No,No,NaN
98,99,5,7,M,1,0,3.82,1240,Math & Science,No,No,NaN


In [ ]:
total_datos = XDB.size
print(f"El número total de datos en el dataframe es: {total_datos}")

El número total de datos en el dataframe es: 208068


Se eliminan los datos atipicos y se seleccionan las variables que necesitamos.

In [ ]:
#Elimino Datos atipicos
XDB=XDB.dropna()
print('Número total de datos después de limpieza:', len(XDB))

Número total de datos después de limpieza: 1528


In [ ]:
XDB["Gender"] = XDB["Gender"].map({"F": 0, "M": 1})
XDB["Admitted"] = XDB["Admitted"].map({"No": 0, "Yes": 1})

# Seleccionar las variables
XD = XDB[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
YD = XDB[['Admitted']]

se proporcionará una síntesis de las variables escogidas para el análisis. Es esencial destacar que la base de datos cuenta con 1528 registros luego de la depuración y que los estudiantes serán evaluados según siete variables:

- Edu_Parent1: Nivel educativo alcanzado por el primer padre o tutor.

- Edu_Parent2: Nivel educativo alcanzado por el segundo padre o tutor.

- Gender: Género del estudiante. El valor 0 representa femenino y el valor 1 masculino.

- White: Señala si el estudiante se identifica como blanco. 0 corresponde a 'NO' y 1 a 'SÍ'.

- Asian: Señala si el estudiante se identifica como asiático. 0 corresponde a 'NO' y 1 a 'SÍ'.

- HSPGA: Promedio de calificaciones del estudiante en la secundaria.

- SAT/ACT: Puntaje obtenido en los exámenes estandarizados SAT o ACT.

Finalmente, la variable de salida será:

- Admitted: Indica si el estudiante es admitido o no en la universidad.

Se procede con la implementación del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4) #El 4 por que son las variables que tengo.
mar.fit(XD,YD)

#Se procede con la grafica
from sklearn.tree import export_graphviz #Me exporta a PNG, JPEG
from pydotplus import graph_from_dot_data #Me crea la grafica

ve= ['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']  #Nombres del Árbol
dot_data=export_graphviz(mar,feature_names=ve)#creeme el arbol y pongale los nombres
graph_from_dot_data(dot_data).write_png('2. ArbolDeDecisión_M.png')

True

In [ ]:
num_nodos_puros = sum(mar.tree_.impurity == 0)
print(f"El número de nodos puros en el árbol es {num_nodos_puros}  (Gini = 0,0):")

El número de nodos puros en el árbol es 7  (Gini = 0,0):


In [ ]:
num_admitidos = YD['Admitted'].sum()  # Sum the values in the 'Admitted' column
print(f"El número de estudiantes admitidos es {num_admitidos}")

El número de estudiantes admitidos es 1303


Se procede con el pronóstico de la admisión para los estudiantes en la base de datos

In [ ]:
XDB0=pd.read_excel(nxl,sheet_name=0)
XDB0.head(100)
#Vamos a clasificar los que estan en la hoja 1 con los solicinates de credito de la hoja 2
#Se crea la base de datos a pronosticar
XDB0["Gender"] = XDB0["Gender"].map({"F": 0, "M": 1})
XD0=XDB0[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
YD0=XDB0[['Admitted']]
filas = XDB0[(XDB0['Admitted'] == 'No') | (XDB0['Admitted'].isna())].index
XD0=np.array(XD0)
XD0=XD0[filas]
#le calculamos la admisión a los estudiantes
y_pred=mar.predict(XD0)
print(y_pred)

#se crea el informe en excel
df=pd.DataFrame(np.column_stack((filas, XD0,y_pred)))
#filas donde estan los estudiantes que no fueron admitidos, XD0 los datos de esas personas, y_pred la decisión
df.columns=['Solicitante', 'Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT','Admitted']
df.to_excel("Admisión_universidad.xlsx")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


[1 1 1 ... 0 1 0]


Evaluar el comportamiento del modelo

In [ ]:
ydp=mar.predict(XD) #pronóstico sobre la bd original
cm=confusion_matrix(YD,ydp)
print("La matriz de confusión es:\n",cm)
VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]


#Métricas de desempeño
EX=(VP+VN)/(VP+VN+FP+FN) #Exactitud
print('La exactitud del modelo es:\n',EX)

TE=(FN+FP)/(VP+VN+FP+FN) #Tasa de error:
print('La tasa de error del modelo es:\n',TE)

SE=VP/(VP+FN) #Sensibilidad
print('La sensibilidad del modelo es:\n',SE)

SP=VN/(VN+FP) #Especificidad
print('La especificidad del modelo es:\n',SP)

PR=VP/(VP+FP) #Precisión
print('La precisión del modelo es:\n',PR)

PRNeng=VN/(VN+FN) #Precisión Negativa
print('La precisión negativa del modelo es:\n',PRNeng)

La matriz de confusión es:
 [[  20  205]
 [   2 1301]]
La exactitud del modelo es:
 0.8645287958115183
La tasa de error del modelo es:
 0.1354712041884817
La sensibilidad del modelo es:
 0.9984650805832693
La especificidad del modelo es:
 0.08888888888888889
La precisión del modelo es:
 0.8638778220451527
La precisión negativa del modelo es:
 0.9090909090909091


#**Análisis de Resultados y del Modelo**

La matriz de confusión revela una mayoría de verdaderos positivos (1302) y falsos positivos (200), con un total de 1528 datos tras la eliminación de valores atípicos. Se identificaron correctamente 1303 estudiantes admitidos y 225 no admitidos, lo que indica un sólido desempeño del modelo.

La exactitud del 86.45% demuestra una alta capacidad para predecir correctamente la admisión o no admisión. Además, la sensibilidad del 99.84% destaca que casi todos los admitidos fueron correctamente identificados, y la precisión del 86.38% refuerza la fiabilidad del modelo en sus predicciones.

Si bien la especificidad del 8.89% sugiere que algunos estudiantes no admitidos fueron clasificados erróneamente, el modelo sigue mostrando un gran rendimiento general. Con ajustes en los umbrales de decisión o el balance de clases, podría mejorar aún más sin afectar su capacidad para predecir de manera efectiva las admisiones.

El modelo tiene un excelente desempeño para identificar a los estudiantes admitidos, pero tiene dificultades para reconocer correctamente a los no admitidos. Esto podría representar un problema si el objetivo es tener un balance entre ambas clasificaciones. En un contexto de admisión universitaria, esta falta de especificidad podría derivar en la aceptación de estudiantes que en realidad no deberían haber sido admitidos.